<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi segundo modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Juan Andrade &bull; 28/11/2025
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este cuaderno está diseñado practicar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [1]:
# Paquetes necesarios
import numpy as np  # Para generación de datos y operaciones numéricas
import pandas as pd  # Para manejo de datos estructurados
import matplotlib.pyplot as plt  # Para visualización de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.linear_model import LinearRegression  # Para el modelo de regresión lineal
from sklearn.metrics import mean_squared_error, r2_score  # Para evaluación del modelo
from sklearn.model_selection import train_test_split

---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv')
#Elegimos solo las variables de estudio:
df_modelo = df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']]
df_modelo.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,221900.0,3,1.00,1180,5650,1.0
1,538000.0,3,2.25,2570,7242,2.0
2,180000.0,2,1.00,770,10000,1.0
3,604000.0,4,3.00,1960,5000,1.0
4,510000.0,3,2.00,1680,8080,1.0


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza un escalado de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [4]:
#Dividimos los datos entre caracteristicas y etiqueta
X = df_modelo[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors']]
y = df_modelo[['price']] #doble corchete para que salga correctamente el .shape()
#Realizamos un sondeo de las variables para ver como podemos proceder con el escalado:
display(X.describe())

,bedrooms,bathrooms,sqft_living,sqft_lot,floors
count,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000
mean,3.370842,2.114757,2079.899736,1.510697e+04,1.494309
std,0.930062,0.770163,918.440897,4.142051e+04,0.539989
min,0.000000,0.000000,290.000000,5.200000e+02,1.000000
25%,3.000000,1.750000,1427.000000,5.040000e+03,1.000000
50%,3.000000,2.250000,1910.000000,7.618000e+03,1.500000
75%,4.000000,2.500000,2550.000000,1.068800e+04,2.000000
max,33.000000,8.000000,13540.000000,1.651359e+06,3.500000


In [5]:
#De acuerdo a esto, vemos que sqft_living y sqft_lot tienen una desviación estándar muy grande a comparación de los otros datos.
#Dada esta dispersión se ve importante reducirla y la manera más efectiva es mediante una estandarización
#(reducimos escala pero evitamos sobrerepresentacion de datos)
st_scaler = StandardScaler()
X_scaled = pd.DataFrame(st_scaler.fit_transform(X), columns=X.columns)
#ahora vemos los valores:
print("Normalizacion por estandarización")
display(X_scaled.head())
#Para comparabilidad realizamos los otros dos métodos (por xmax y por xmax xmin)
# Normalización por el máximo
x_max = X.max()
X_max_normalized = X / x_max
print("Normalización por el máximo:\n")
display(X_max_normalized.head())

# Normalización Min-Max
min_max_scaler = MinMaxScaler()
X_min_max_normalized = pd.DataFrame(min_max_scaler.fit_transform(X), columns=X.columns)
print("Normalización Min-Max:\n")
display(X_min_max_normalized.head())

Normalizacion por estandarización


,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.398737,-1.447464,-0.979835,-0.228321,-0.915427
1,-0.398737,0.175607,0.533634,-0.189885,0.936506
2,-1.473959,-1.447464,-1.426254,-0.123298,-0.915427
3,0.676485,1.149449,-0.130550,-0.244014,-0.915427
4,-0.398737,-0.149007,-0.435422,-0.169653,-0.915427


Normalización por el máximo:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.087149,0.003421,0.285714
1,0.090909,0.28125,0.189808,0.004385,0.571429
2,0.060606,0.12500,0.056869,0.006056,0.285714
3,0.121212,0.37500,0.144756,0.003028,0.285714
4,0.090909,0.25000,0.124077,0.004893,0.285714


Normalización Min-Max:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.090909,0.12500,0.067170,0.003108,0.0
1,0.090909,0.28125,0.172075,0.004072,0.4
2,0.060606,0.12500,0.036226,0.005743,0.0
3,0.121212,0.37500,0.126038,0.002714,0.0
4,0.090909,0.25000,0.104906,0.004579,0.0


Se puede observar que algunas variables no cambian entre si se hacen solo con el máximo o con el metodo min max, y aquellas que mencionamos son las que más cambiaron por su dispersión. Dado que parece que estan mejor distribuidos los valores con la estandarización usaremos esto sin estandarizar y para que nos entregue el modelo los valores y reales.

In [6]:
#Este apartado es para completar los tipos de normalización pedidos para comparar sus errores cuadráticos medios:
#Robust Scaler
robust_scaler = RobustScaler()
X_robust_normalized = pd.DataFrame(robust_scaler.fit_transform(X), columns=X.columns)
print("Normalización Robust:\n")
display(X_robust_normalized.head())
#Escalado tipo normalizer
normalizer = Normalizer()
X_normalized = pd.DataFrame(normalizer.fit_transform(X), columns=X.columns)
print("Normalización tipo normalizer:\n")
display(X_normalized.head())
#Escalado QuantileTransformer
quantile_transformer = QuantileTransformer()
X_quantile_normalized = pd.DataFrame(quantile_transformer.fit_transform(X), columns=X.columns)
print("Normalización QuantileTransformer:\n")
display(X_quantile_normalized.head())
3#Escalado PowerTransformer
power_transformer = PowerTransformer()
X_power_normalized = pd.DataFrame(power_transformer.fit_transform(X), columns=X.columns)
print("Normalización PowerTransformer:\n")
display(X_power_normalized.head())

Normalización Robust:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.0,-1.666667,-0.650045,-0.348442,-0.5
1,0.0,0.000000,0.587711,-0.066572,0.5
2,-1.0,-1.666667,-1.015138,0.421742,-0.5
3,1.0,1.000000,0.044524,-0.463527,-0.5
4,0.0,-0.333333,-0.204809,0.081799,-0.5


Normalización tipo normalizer:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.000520,0.000173,0.204438,0.978879,0.000173
1,0.000390,0.000293,0.334440,0.942417,0.000260
2,0.000199,0.000100,0.076773,0.997049,0.000100
3,0.000745,0.000559,0.364961,0.931022,0.000186
4,0.000364,0.000242,0.203567,0.979061,0.000121


Normalización QuantileTransformer:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,0.366867,0.093594,0.138639,0.304304,0.00000
1,0.366867,0.527027,0.758258,0.455122,0.77027
2,0.074575,0.093594,0.016016,0.713213,0.00000
3,0.753253,0.894895,0.520020,0.238739,0.00000
4,0.366867,0.433934,0.383383,0.554555,0.00000


Normalización PowerTransformer:



,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,-0.352407,-1.571853,-1.121655,-0.331682,-0.975610
1,-0.352407,0.250882,0.706456,-0.032289,1.029432
2,-1.605435,-1.571853,-2.107060,0.336051,-0.975610
3,0.722633,1.136536,0.065492,-0.484436,-0.975610
4,-0.352407,-0.072697,-0.296964,0.095276,-0.975610


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada elemento.

In [7]:
#Antes de realizar el escalado ya se hizo la separación. Por tanto, mostraremos la dimension de cada elemento
print("Dimension de X: ", X.shape)
print("Dimension de y: ", y.shape)

Dimension de X:  (21613, 5)
Dimension de y:  (21613, 1)


---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

In [8]:
#mi cédula termina en 1 por tanto esa sera la semilla para el modelo lineal (aqui cambiaremos para ver resultados de acuerdo a las distintas normalizaciones)
X_train, X_test, y_train, y_test = train_test_split(X_power_normalized, y, test_size=0.2, random_state=1) #establecemos primero un 20% de datos de testeo

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

In [9]:
# Reviso las dimensiones de los conjuntos de datos
print('Dimensiones de los conjuntos de datos:')
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

Dimensiones de los conjuntos de datos:
X_train: (17290, 5)
y_train: (17290, 1)
X_test: (4323, 5)
y_test: (4323, 1)


Se puede observar que las etiquetas se han dividido como sigue:
* Usaremos 17290 etiquetas para los datos de entrenamiento.
* Usaremos 4323 etiquetas para testeo.

---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y entrenalo con los datos de entrenamiento.

In [10]:
# Crear el modelo y ajustarlo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Visualizar los coeficientes
print('Ecuación de la recta\n')

print("y =", modelo.coef_[0][0], "* x1 +",
    modelo.coef_[0][1], "* x2 +",
    modelo.coef_[0][2], "* x3 +",
    modelo.coef_[0][3], "* x4 +",
    modelo.coef_[0][4], "* x5 +",
    modelo.intercept_[0])

Ecuación de la recta

y = -53834.72884347378 * x1 + 18282.687921957113 * x2 + 248711.45560798855 * x3 + -18163.822280840424 * x4 + -5504.058026781375 * x5 + 538625.4346771163


---
## <font color='264CC7'> Evaluación </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

In [ ]:
# Predicciones del modelo
y_pred = modelo.predict(X_test)
# Comparar las predicciones con los datos de prueba
df_predicciones = pd.DataFrame({
    'Actual': y_test.values.flatten(),
    'Predicción': y_pred.flatten()
})
display(df_predicciones)

,Actual,Predicción
0,459000.0,769353.511343
1,445000.0,444350.515417
2,1057000.0,716139.084999
3,732350.0,643151.574917
4,235000.0,354556.158805
...,...,...
4318,965000.0,834554.021909
4319,359950.0,677075.886360
4320,260000.0,489711.263001
4321,1795000.0,873967.782793


Calcula el error cuadrático medio.

In [ ]:
# Métricas de evaluación
mse = mean_squared_error(y_test, y_pred)

print('Error cuadrático medio:', mse)

Error cuadrático medio: 122519071402.01677


Escribe el error cuadrático medio para cada tipo de escalado:

- Escalado MaxAbsScaler: 82655267239.84705
- Escalado MinMaxScaler: 82655267239.84706
- Escalado RobustScaler: 82655267239.84703
- Escalado StandardScaler: 82655267239.84705
- Escalado Normalizer: 139311582675.88434
- Escalado QuantileTransformer: 122519071402.01677
- Escalado PowerTransformer: 122519071402.01677

PD: para esto únicamente se cambio en el modelo el tipo de normalización de x usado y se volvio a correr el código.

Escribe un párrafo sobre qué hace cada escalado no visto en clase:

- **Escalado Robusto:**
El escalado Robusto, de acuerdo a la guía de Sckit-Learn, es un método que escala con estadísticas robustas a valores atípicos, removiendo la mediana y calculando según el rango intercuartílico (RIC). La fórmula es:
$$x_{scaled} = \frac{x - Q_2(x)}{Q_3(x) - Q_1(x)}$$

- **Escalado Normalizer:**
De acuerdo a la guía Sckit-Learn, el Normalizaer trabaja en las filas (es decir en las muestras y no en las variables) llevando a los vectores a su norma unitaria. La norma usada por defecto es la norma euclidiana, y la fórmula del normalizador es:
$$x_{scaled} = \frac{x}{\sqrt{x_1^2 + x_2^2 + ... + x_n^2}}$$

- **Escalado por Transformación Cuantil:**
Es otro tipo de preprocesamiento robusto que utiliza información de cuantiles. La guía menciona que la transformación tiende a dispersar los valores más frecuentes y busca que los datos sigan distribuciones uniformes o normales. Algo importante que menciona es que la transformación no es lineal y, al alterar el espacio, puede rompe las correlaciones lineales entre datos, pero mejorar comparabilidad en datos con escalas distintas.

- **Escalado por Transformación de Potencia:**
Citando a la guía de Sckit-Learn sobre este tipo de Escalado, dice que "Las transformaciones de potencia son una familia de transformaciones paramétricas y monótonas que se aplican para que los datos tengan una mayor similitud con la gaussiana." (desarrolladores de scikit-learn, s.f.) Esto es especialmente útil cuando no existen varianzas constantes o buscamos en sí normalidad. La forma que usa para estabilizar lo dicho anteriormente es mediante máxima verosimilitud y las transformadas admitidas son Box-Cox y yeo jhonson (esta última siendo la transformación por defecto en el preprocesamiento de PowerTransformer).

**Fuentes Bibliográficas**

GeeksforGeeks. (2025, 23 julio). Difference Between StandardScaler and Normalizer in sklearn.preprocessing. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/difference-between-standardscaler-and-normalizer-in-sklearn-preprocessing/

Google. (2025). Gemini (Versión del 28 de noviembre) [Gemini 3 Pro]. https://gemini.google.com/

Normalizer. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

PowerTransformer. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html

QuantileTransformer. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html

RobustScaler. (s. f.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html